# Optimización de modelos GPT-2 para generación de poesía y texto en español

# Integrantes:

* Yesid Castelblanco
* Albin Rivera

# Introduccion

El presente informe documenta la implementación de un modelo de generación de texto en español, desarrollado y ejecutado en un entorno Kaggle Notebook. El objetivo principal es explorar las capacidades de los modelos de lenguaje de última generación para producir texto creativo y coherente, en este caso con énfasis en composiciones poéticas. La metodología aplicada incluye la preparación del entorno de trabajo, la instalación de librerías con versiones específicas, la carga y limpieza del conjunto de datos, la tokenización del corpus, el ajuste del modelo y la validación de los resultados a través de ejemplos generados. Para este ejercicio se utilizó el Spanish Poetry Dataset, disponible en la plataforma Hugging Face y recopilado en el marco del Somos NLP Hackathon 2022. Este conjunto de datos contiene una amplia colección de poemas en idioma español que abarca diferentes autores, estilos y épocas. 

# Dependencias y entorno

A continuación, se lleva a cabo la configuración inicial del entorno de trabajo en Kaggle. En él se definen las dependencias necesarias y se garantiza la reproducibilidad de la ejecución. Primero, se implementa una función auxiliar que permite instalar paquetes con versiones específicas, lo que asegura estabilidad y compatibilidad entre librerías. A través de esta función se instalan las bibliotecas clave para el proyecto como transformers, datasets y accelerate. Asimismo, se desinstala peft para prevenir conflictos en el entorno. Luego, se importan módulos generales de Python y librerías propias de aprendizaje profundo como torch, datasets y transformers, empleadas para el manejo de datos y la construcción del modelo de lenguaje. Posteriormente, se configura un directorio local de caché en la ruta de trabajo de Kaggle (/kaggle/working/hf_cache), con el fin de almacenar los recursos de Hugging Face y facilitar ejecuciones posteriores sin necesidad de descargas repetidas. Finalmente, se incorpora una verificación del hardware disponible para determinar si el entrenamiento podrá realizarse en GPU, mostrando en caso afirmativo el nombre de la tarjeta gráfica detectada. 

In [1]:
# ============================
# 1) DEPENDENCIAS Y ENTORNO
# ============================
import os, sys, subprocess

def install_package(package):
    subprocess.run([sys.executable, "-m", "pip", "install", package, "--quiet", "--no-warn-conflicts"])

# Instalar dependencias clave
install_package("transformers==4.44.2")
install_package("datasets==3.0.1")
install_package("accelerate==0.30.1")

subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "peft"])

# Imports base
import warnings
warnings.filterwarnings("ignore")

import torch
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments,
    DataCollatorForLanguageModeling
)

# Configuración de caché en Kaggle
cache_dir = "/kaggle/working/hf_cache"
os.makedirs(cache_dir, exist_ok=True)
os.environ["HF_HOME"] = cache_dir

# Verificar GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Ejecutando en:", device)
if device == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

2025-09-21 19:29:10.691373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758482950.848422      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758482950.893137      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Ejecutando en: cuda
GPU: Tesla P100-PCIE-16GB


Lo anterior indica que la instalación de las dependencias clave se realizó de manera exitosa, fijando las versiones de transformers, datasets y accelerate, y desinstalando correctamente el paquete peft para evitar conflictos. Asimismo, la verificación de hardware confirmó que la ejecución se llevará a cabo en una GPU Tesla P100-PCIE-16GB, lo cual asegura la disponibilidad de aceleración por hardware para las etapas de entrenamiento y generación de texto.

# Inicialización del tokenizer y modelo con configuración de special tokens

Se define la función init_model_and_tokenizer, cuyo propósito es inicializar el modelo de lenguaje y su tokenizador, garantizando que ambos queden configurados de manera consistente para el entrenamiento y la generación de texto. En primer lugar, la función carga el tokenizador y el modelo especificado a partir de la librería Hugging Face y los ubica en el dispositivo de cómputo disponible (CPU o GPU). Posteriormente, se asegura de que los tokens especiales estén correctamente definidos, es decir si alguno de los tokens de padding (pad), inicio (bos) o fin de secuencia (eos) no existe, se asigna un valor por defecto para evitar errores durante el proceso de entrenamiento o de inferencia. Una vez establecidos, estos identificadores se sincronizan dentro de la configuración del modelo, alineando la codificación del tokenizador con la arquitectura de la red neuronal. Finalmente, en caso de que se hayan agregado nuevos tokens especiales, se redimensionan las capas de embeddings del modelo para que correspondan al tamaño actualizado del vocabulario. La función imprime en pantalla los tokens configurados y retorna tanto el modelo como el tokenizador listos para ser utilizados en las siguientes etapas del flujo de trabajo.

In [2]:
def init_model_and_tokenizer(model_name, device="cuda"):
    
    # 1. Cargar tokenizer y modelo
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

    # 2. Asegurar tokens especiales
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
    if tokenizer.bos_token is None:
        tokenizer.add_special_tokens({"bos_token": tokenizer.eos_token})
    if tokenizer.eos_token is None:
        tokenizer.add_special_tokens({"eos_token": ""})  # fallback

    # 3. Alinear configuración del modelo
    model.config.pad_token_id = tokenizer.pad_token_id
    model.config.eos_token_id = tokenizer.eos_token_id
    model.config.bos_token_id = tokenizer.bos_token_id

    # 4. Redimensionar embeddings si se añadieron nuevos tokens
    model.resize_token_embeddings(len(tokenizer))

    print("✅ Modelo y tokenizer listos")
    print("pad_token:", tokenizer.pad_token, tokenizer.pad_token_id)
    print("bos_token:", tokenizer.bos_token, tokenizer.bos_token_id)
    print("eos_token:", tokenizer.eos_token, tokenizer.eos_token_id)

    return model, tokenizer

A continuación, se lleva a cabo la carga y depuración del conjunto de datos de poesía en español utilizado para entrenar el modelo. En primer lugar, se accede al dataset Spanish Poetry Dataset desde Hugging Face mediante la función load_dataset, especificando la ruta de caché previamente configurada para optimizar la descarga y almacenamiento local en Kaggle. Posteriormente, se realiza una verificación para identificar registros nulos o vacíos en la columna principal (content), que contiene los poemas. Una vez identificados, se filtran los textos inválidos con el fin de conservar únicamente aquellos que contienen información relevante.

Tras este filtrado inicial, se imprime en consola el tamaño del dataset antes y después de la limpieza, lo que permite constatar la reducción en el número de registros y la mejora en la calidad del corpus. Como medida adicional de validación, los datos se convierten a un DataFrame de pandas, donde se eliminan de forma explícita los valores nulos y las cadenas vacías que pudieran permanecer. Finalmente, el dataset limpio se reconstruye en un formato compatible con Hugging Face (DatasetDict), quedando disponible para las siguientes etapas de tokenización y preparación del entrenamiento.

In [3]:
# ============================
# 2) CARGAR DATASET DE POESÍA
# ============================
dataset = load_dataset("somosnlp-hackathon-2022/spanish-poetry-dataset", cache_dir=cache_dir)

# Verificar cuántos textos nulos o vacíos hay en 'content'
mask = [text is None or text.strip()=="" for text in dataset["train"]["content"]]
print("Nulos o vacíos:", sum(mask), "de", len(mask))

# Filtrar solo filas con contenido válido
ds_clean = dataset["train"].filter(lambda x: x["content"] is not None and x["content"].strip() != "")

print(f"Antes: {len(dataset['train'])}, Después filtrado: {len(ds_clean)}")

# Convertir a pandas para asegurarse de eliminar restos de NA
df_clean = ds_clean.to_pandas().dropna(subset=["content"])
df_clean = df_clean[df_clean["content"].str.strip() != ""]

# Reconstruir dataset HuggingFace
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_clean, preserve_index=False)
})

print(dataset)

README.md:   0%|          | 0.00/196 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


poems.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/5133 [00:00<?, ? examples/s]

Nulos o vacíos: 6 de 5133


Filter:   0%|          | 0/5133 [00:00<?, ? examples/s]

Antes: 5133, Después filtrado: 5127
DatasetDict({
    train: Dataset({
        features: ['author', 'content', 'title'],
        num_rows: 5127
    })
})


La ejecución de la carga y limpieza del dataset arrojó como resultado un total de 5.133 registros iniciales, de los cuales se identificaron 6 entradas nulas en la columna de contenido. Tras aplicar los filtros correspondientes para eliminar estos registros inválidos, el conjunto de datos quedó conformado por 5.127 poemas válidos. Finalmente, el dataset se reconstruyó en formato DatasetDict de Hugging Face, manteniendo las tres variables originales siendo estas autor, contenido y título. De esta forma, el corpus depurado quedó listo para su uso en las siguientes fases de tokenización y entrenamiento del modelo de generación de texto. 

Se procede a transformar el dataset previamente cargado y limpiado a un formato pandas DataFrame, lo cual facilita la manipulación y el análisis exploratorio de los datos. Los resultados muestran que el dataset está compuesto por tres columnas principales siendo estas author (nombre del autor del poema), content (texto del poema) y title (título de la obra). En la salida se observan fragmentos de poemas de diversos autores, entre ellos Leopoldo Lugones, Gabriela Mistral, Antonio Colinas y William Shakespeare. Esta visualización confirma tanto la validez del corpus como la diversidad de estilos y épocas que lo conforman, evidenciando que se cuenta con un recurso literario amplio y variado para el entrenamiento del modelo de generación de texto.


In [4]:
dataset.set_format('pandas')
df = dataset['train'].to_pandas()
df.head(10)

,author,content,title
0,Leopoldo Lugones,\n\nEn el parque confuso\nQue con lánguidas br...,LA MUERTE DE LA LUNA
1,Marilina Rébora,"\n\nPorque si tú no velas, vendré como ladrón;...",PORQUE SI TÚ NO VELAS
2,Antonio Colinas,"\n\nPequeña de mis sueños, por tu piel las pal...",POEMA DE LA BELLEZA CAUTIVA QUE PERDÍ
3,José María Hinojosa,\n\nLos dedos de la nieve\nrepiquetearon\nen e...,SENCILLEZ
4,Rubén Izaguirre Fiallos,"Naciste en Armenia,\npero te fuiste a vivir al...",Breve Carta a Consuelo Suncín
5,Leopoldo María Panero,\n\nOscuridad nieve buitres desespero oscurida...,PASADIZO SECRETO
6,Gabriela Mistral,\nSiento mi corazón en la dulzura \nfundirse c...,Atardecer
7,Pablo Neruda,Cien sonetos de amor\n\nTrajo el amor su cola ...,Cien sonetos de amor
8,William Shakespeare,¿Y por qué no es tu guerra más pujante\ncontra...,Y por qué no es tu guerra más pujante...
9,Gabriela Mistral,\nEl espino prende a una roca \nsu enloquecida...,El espino


A continuación,se calcula la extensión de cada poema en número de palabras para caracterizar el corpus de manera cuantitativa. Para ello, se crea la columna Palabras_por_poesia, donde cada valor corresponde al conteo de palabras en el texto del campo content. Una vez generado este indicador, se obtiene la mediana de longitud de los poemas, la cual resultó en 107 palabras por composición. Este valor sirve como referencia estadística sobre la extensión típica de los textos, información clave para definir parámetros posteriores en la tokenización y entrenamiento del modelo.

In [5]:
df['Palabras_por_poesia'] = (
    df['content']
    .fillna("")
    .str.split()
    .apply(len)
)

print(f"La mediana de palabras por poesía es: {df['Palabras_por_poesia'].median()} palabras")

La mediana de palabras por poesía es: 107.0 palabras


Ya con lo anterior se procede a ejecutar la tokenización y partición del dataset para preparar los datos para el entrenamiento del modelo. Primero, se define la función preprocess_function, la cual utiliza el tokenizador previamente configurado para transformar los poemas en secuencias de identificadores numéricos (input_ids). En este paso, se establece una longitud máxima de secuencia de 256 tokens, aplicando truncamiento en los textos más largos y padding en los más cortos, con el fin de homogeneizar las entradas. Luego, se aplica esta función a todo el dataset con .map(), obteniendo como salida un conjunto tokenizado en el que se conservan únicamente las columnas necesarias. Posteriormente, se realiza una división en subconjuntos de entrenamiento y prueba, destinando el 90% de los datos al entrenamiento y el 10% restante a la evaluación. Finalmente, el dataset se configura en formato torch para que pueda ser utilizado directamente por el framework PyTorch durante el entrenamiento.

In [7]:
def preprocess_function(max_len):
    def _preprocess_function(examples):
        return tokenizer(examples['content'], max_length=max_len, truncation=True, padding='max_length')
    return _preprocess_function
model_name = "datificate/gpt2-small-spanish"
model, tokenizer = init_model_and_tokenizer(model_name, device=device)
dataset.reset_format()
tokenized_dataset = dataset['train'].map(preprocess_function(max_len=256), batched=True)
tokenized_dataset = tokenized_dataset.remove_columns([col for col in tokenized_dataset.column_names if col != 'input_ids'])
tokenized_dataset = tokenized_dataset.train_test_split(train_size=0.9)
tokenized_dataset.set_format('torch')
tokenized_dataset

tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

✅ Modelo y tokenizer listos
pad_token: <|endoftext|> 0
bos_token: <|endoftext|> 0
eos_token: <|endoftext|> 0


Map:   0%|          | 0/5127 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 4614
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 513
    })
})

Los resultados muestran que, de un total de 5.127 poemas válidos, se asignaron 4.614 registros al conjunto de entrenamiento y 513 al de prueba, ambos conteniendo únicamente la representación numérica de los textos en la forma de la columna input_ids. 

# Validacion rapida para saber a que modelo darle mas tiempo

Se procede a realizar una evaluación comparativa de distintos modelos de lenguaje en español y en inglés adaptados al castellano, entrenándolos de manera rápida sobre un subconjunto reducido de datos. Para cada modelo se registró el valor de loss, el tiempo de entrenamiento y un ejemplo de texto generado a partir del prompt “En el silencio de la noche”. Los resultados se organizaron en una tabla que facilitó la comparación entre las alternativas, y a partir de este análisis se seleccionó automáticamente el modelo con menor. De esta manera se identifico que el modelo que arrojo mejor desempeño y capacidad generativa en la validación fue datificate/gpt2-small-spanish", "gpt2-small-es.

In [8]:

import time
import pandas as pd
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    Trainer, TrainingArguments, DataCollatorForLanguageModeling
)



# ============================
# FUNCIÓN DE EVALUACIÓN
# ============================
def evaluar_modelo(model_name, dataset, nombre=None, epochs=1, batch_size=4):
    
    print(f"\n=== Entrenando {nombre or model_name} ===")

    # Tokenizador y modelo
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

    # Argumentos de entrenamiento
    training_args = TrainingArguments(
        output_dir=f"./{nombre}-out",
        overwrite_output_dir=True,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        evaluation_strategy="steps",
        eval_steps=100,
        logging_steps=50,
        save_strategy="no",        # no guardamos checkpoints en pruebas
        report_to="none",
        disable_tqdm=True,
        fp16=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
        train_dataset=dataset["train"].select(range(500)),  # subset para prueba rápida
        eval_dataset=dataset["test"].select(range(100)),
        tokenizer=tokenizer
    )

    start = time.time()
    result = trainer.train()
    end = time.time()

    tiempo = end - start
    loss = result.training_loss

    # Generación de ejemplo corto
    prompt = "En el silencio de la noche"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=60,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )
    ejemplo = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return {
        "Modelo": nombre or model_name,
        "Loss": round(loss, 4),
        "Tiempo (seg)": round(tiempo, 2),
        "Ejemplo": ejemplo[:200]  # recortamos
    }

# ============================
# LISTA DE MODELOS A PROBAR
# ============================
modelos = [
    ("distilgpt2", "distilgpt2"),
    ("flax-community/gpt-2-spanish", "gpt-2-spanish"),
    ("mrm8488/spanish-gpt2", "spanish-gpt2"),
    ("datificate/gpt2-small-spanish", "gpt2-small-es"),
    ("ITG/DialoGPT-medium-spanish-chitchat", "DialoGPT-medium-spanish-chitchat"),
    ("ostorc/Conversational_Spanish_GPT", "Conversational_Spanish_GPT"),
    ("EleutherAI/gpt-neo-125M", "gpt-neo-125M")
]


# ============================
# EJECUTAR EXPERIMENTOS
# ============================
resultados = []
for model_id, nombre in modelos:
    res = evaluar_modelo(model_id, tokenized_dataset, nombre=nombre, epochs=1, batch_size=4)
    resultados.append(res)

# Mostrar resultados
df_resultados = pd.DataFrame(resultados)
print(df_resultados)

# ============================
# SELECCIÓN AUTOMÁTICA DEL GANADOR
# ============================
ganador = df_resultados.sort_values("Loss").iloc[0]
print("\n=== MODELO RECOMENDADO ===")
print(f"Modelo: {ganador['Modelo']} | Loss: {ganador['Loss']} | Tiempo: {ganador['Tiempo (seg)']} seg")



=== Entrenando distilgpt2 ===


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

{'loss': 4.9161, 'grad_norm': 3.304161548614502, 'learning_rate': 3.04e-05, 'epoch': 0.4}
{'loss': 4.7054, 'grad_norm': 1.957353115081787, 'learning_rate': 1.04e-05, 'epoch': 0.8}
{'eval_loss': 4.536550998687744, 'eval_runtime': 1.4953, 'eval_samples_per_second': 66.874, 'eval_steps_per_second': 16.719, 'epoch': 0.8}


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'train_runtime': 24.9162, 'train_samples_per_second': 20.067, 'train_steps_per_second': 5.017, 'train_loss': 4.708394409179688, 'epoch': 1.0}
                             Modelo    Loss  Tiempo (seg)  \
0                        distilgpt2  5.3409         16.16   
1                     gpt-2-spanish  5.6335         23.60   
2                      spanish-gpt2  6.0712         23.63   
3                     gpt2-small-es  4.4004         23.86   
4  DialoGPT-medium-spanish-chitchat  5.7679         62.31   
5        Conversational_Spanish_GPT  6.9103         24.00   
6                      gpt-neo-125M  4.7084         25.42   

                                             Ejemplo  
0  En el silencio de la noche to o l- a a to u o ...  
1  En el silencio de la noche, la voz de la noche...  
2  En el silencio de la nocheLa música de amor qu...  
3  En el silencio de la noche, una mujer lloraba ...  
4  En el silencio de la noche pregúre h to ehe an...  
5  En el silencio de la noche de todo 

# Entrenamiento del modelo con un solo dataset

Ya con la selección del modelo, se llevó a cabo el entrenamiento, utilizando el corpus de poesía previamente tokenizado. Para ello, se configuraron los parámetros de ajuste fino con 20 épocas de entrenamiento, batch size de 32, tasa de aprendizaje de 3e-5 y estrategias de evaluación y guardado cada 200 pasos. Se incluyó un callback personalizado que calculó y mostró la perplejidad (PPL) junto con la pérdida de validación, lo que permitió monitorear en tiempo real la capacidad del modelo para predecir secuencias de palabras. Con esta configuración, el proceso de entrenamiento optimizó de manera progresiva los parámetros del modelo, garantizando un mejor ajuste al estilo y estructura propios de los poemas en español.

In [ ]:
from transformers import TrainerCallback
import math

# ============================
# CALLBACK PERSONALIZADO
# ============================
class PerplexityCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics and "eval_loss" in metrics:
            ppl = math.exp(metrics["eval_loss"])
            print(f"\n🔎 Step {state.global_step}: Eval Loss = {metrics['eval_loss']:.4f} | PPL = {ppl:.2f}")

# ============================
# 3) TOKENIZADOR Y MODELO BASE
# ============================
model_name = "datificate/gpt2-small-spanish"
model, tokenizer = init_model_and_tokenizer(model_name, device=device)

batch_size = 32

training_args = TrainingArguments(
    output_dir="./hf-gpt",
    run_name="poesia-gpt2",
    overwrite_output_dir=True,
    num_train_epochs=20,#20
    learning_rate=3e-5,#3e-5, 5e-5
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay=0.05,#0.01
    warmup_steps=40, #1500, 100
    evaluation_strategy="steps",  
    eval_steps=200,               
    save_steps=200,               
    logging_strategy="steps",
    logging_steps=50,
    logging_first_step=True,
    save_strategy="steps",    
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
    disable_tqdm=False,
    fp16=True,
    gradient_accumulation_steps = 2 #Nuevo
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    callbacks=[PerplexityCallback]   # 👈 Agregamos el callback
)

# ============================
# 4) ENTRENAMIENTO
# ============================
trainer.train()



✅ Modelo y tokenizer listos
pad_token: <|endoftext|> 0
bos_token: <|endoftext|> 0
eos_token: <|endoftext|> 0


Step,Training Loss,Validation Loss


In [ ]:
def generar_texto(model, tokenizer, prompt="En el silencio de la noche"):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=120,
        num_return_sequences=5,
        temperature=1.1,
        top_k=30,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id   # 🔑 ya definido arriba
    )
    for i, out in enumerate(outputs):
        print(f"=== Poema {i+1} ===")
        print(tokenizer.decode(out, skip_special_tokens=True))
        print()

generar_texto(model, tokenizer, "Había una vez un poeta que soñaba")

El entrenamiento del modelo gpt2-small-spanish alcanzó valores finales de training loss cercanos a 4.43 y un validation loss de 4.21, lo que se tradujo en una perplejidad (PPL) de 67.36. Posteriormente, se evaluó la capacidad creativa del modelo mediante la función de generación de texto, utilizando como prompt inicial “Había una vez un poeta que soñaba”. Los resultados muestran cinco poemas producidos de manera automática, en los cuales el modelo logró mantener una estructura literaria coherente, evocando temáticas propias de la poesía como el amor, la vida, los recuerdos y la naturaleza; sin embargo, también se evidencian repeticiones y pasajes con baja coherencia semántica, reflejo de las limitaciones del entrenamiento con un corpus reducido. Ante estos resultados, se tomó la decisión de combinar el dataset de poesía con otros conjuntos de datos, con el fin de enriquecer el vocabulario, mejorar la capacidad de generalización y reducir la perplejidad en futuras iteraciones.

# Combinación de  Datasets

Con base en el resultado anterior y la desición posterior, se lleva a cabo la carga y combinación de distintos corpus en español para fortalecer el entrenamiento del modelo de lenguaje. En primer lugar, se descargan tres conjuntos de datos desde Hugging Face, un corpus de poesía en español (Spanish Poetry Dataset), una muestra del Wikipedia en español y un subconjunto de noticias del dataset MLSUM en español. Dado que la ejecución se realiza en una GPU P100, se utilizan únicamente fracciones reducidas de Wikipedia y MLSUM para optimizar recursos. Posteriormente, se aplica un proceso de limpieza de datos, eliminando entradas vacías o nulas, y se normaliza la estructura de los tres corpus bajo una única columna llamada “content”. A continuación, cada dataset se convierte a formato pandas DataFrame, lo que permite concatenarlos en un único corpus unificado. Sobre este conjunto combinado se ejecuta una limpieza final para remover posibles valores faltantes, y luego se reconstruye en formato Dataset de Hugging Face, dividiéndolo en subconjuntos de entrenamiento (90%) y prueba (10%).

El resultado es un corpus integrado que combina poesía, textos enciclopédicos y noticias en español, aportando diversidad léxica y temática. Este enriquecimiento busca mejorar la generalización del modelo y reducir la perplejidad obtenida en entrenamientos previos con un corpus limitado únicamente a textos poéticos.

In [ ]:
# ============================
# 2) CARGA Y COMBINACIÓN DE CORPUS
# ============================
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd

# Dataset de poesía
poesias = load_dataset("somosnlp-hackathon-2022/spanish-poetry-dataset", cache_dir=cache_dir)

# Dataset de Wikipedia en español (usar solo una fracción para GPU P100)
wiki = load_dataset("wikimedia/wikipedia", "20231101.es", split="train[:1%]", cache_dir=cache_dir)

# Dataset de noticias MLSUM en español (usar solo una fracción)
noticias = load_dataset("mlsum", "es", split="train[:1%]", trust_remote_code=True, cache_dir=cache_dir)

# 🔹 Limpieza
def filter_valid(ex):
    text = ex.get("content", ex.get("text", None))
    return text is not None and text.strip() != ""

poesias = poesias["train"].filter(lambda x: x["content"] is not None and x["content"].strip() != "")
wiki = wiki.filter(lambda x: x["text"] is not None and x["text"].strip() != "")
noticias = noticias.filter(lambda x: x["text"] is not None and x["text"].strip() != "")

# 🔹 Normalizar a columna "content"
poesias = poesias.map(lambda x: {"content": x["content"]})
wiki = wiki.map(lambda x: {"content": x["text"]})
noticias = noticias.map(lambda x: {"content": x["text"]})

# 🔹 Convertir a pandas
df_poesias = poesias.to_pandas()[["content"]]
df_wiki = wiki.to_pandas()[["content"]]
df_noticias = noticias.to_pandas()[["content"]]

# 🔹 Concatenar y limpiar
df_comb = pd.concat([df_poesias, df_wiki, df_noticias], ignore_index=True)
df_comb = df_comb.dropna().reset_index(drop=True)

# 🔹 Reconstruir Dataset HuggingFace
dataset = Dataset.from_pandas(df_comb)
dataset = dataset.train_test_split(train_size=0.9, seed=42)

dataset = DatasetDict({
    "train": dataset["train"],
    "test": dataset["test"]
})

print(dataset)
print("Ejemplo:", dataset["train"][0])



La combinación de corpus permitió integrar exitosamente los tres conjuntos de datos en español (poesía, Wikipedia y noticias MLSUM). Tras la descarga y filtrado inicial, se eliminaron los registros vacíos y se homogenizó la estructura bajo la columna “content”. El resultado final fue un DatasetDict con un total de 2.918 registros, de los cuales el 90% (2.626 ejemplos) se destinó al entrenamiento y el 10% (292 ejemplos) a validación. En la impresión de consola se observa además un ejemplo de texto del dataset combinado, correspondiente a una entrada enciclopédica, lo que confirma la correcta integración de las fuentes. Este resultado garantiza un corpus más diverso y robusto, capaz de enriquecer el vocabulario del modelo y mejorar su capacidad de generalización en comparación con el uso exclusivo del corpus poético.

# Tokenización del dataset combinado

Se lleva a cabo la tokenización del dataset combinado y la preparación de los datos para el entrenamiento del modelo. Para ello, se definió una función de preprocesamiento que transforma el contenido textual en secuencias de tokens con una longitud máxima de 256, aplicando truncamiento y padding para homogeneizar las entradas. Una vez aplicado este proceso al conjunto de entrenamiento, se eliminaron las columnas innecesarias conservando únicamente los identificadores de tokens (input_ids). Posteriormente, el dataset resultante se dividió nuevamente en subconjuntos de entrenamiento (90%) y validación (10%), y se configuró en formato compatible con PyTorch.


In [ ]:
# ============================
# 3) TOKENIZADOR Y MODELO
# ============================
def preprocess_function(max_len):
    def _preprocess_function(examples):
        return tokenizer(examples['content'], max_length=max_len, truncation=True, padding='max_length')
    return _preprocess_function

dataset.reset_format()
tokenized_dataset = dataset['train'].map(preprocess_function(max_len=256), batched=True)
tokenized_dataset = tokenized_dataset.remove_columns([col for col in tokenized_dataset.column_names if col != 'input_ids'])
tokenized_dataset = tokenized_dataset.train_test_split(train_size=0.9)
tokenized_dataset.set_format('torch')


Lo anterior confirma la correcta ejecución del mapeo sobre 23.582 ejemplos, con un procesamiento aproximado de 342 ejemplos por segundo. De esta manera, el corpus enriquecido y tokenizado quedó listo para iniciar la fase de ajuste fino del modelo, asegurando tanto la calidad de los datos como su compatibilidad con el framework de entrenamiento.

# Entrenamiento del modelo con el dataset combinado y monitoreo de perplejidad

A continuación se ejecuta el entrenamiento del modelo con el corpus combinado, incorporando el callback de perplejidad para monitorear el desempeño. Se cargó nuevamente el modelo base gpt2-small-spanish junto con su tokenizador, y se configuraron parámetros de ajuste con 20 épocas de entrenamiento, un batch size de 32, tasa de aprendizaje de 3e-5, weight decay de 0.05 y acumulación de gradientes cada 2 pasos para optimizar recursos en GPU. Asimismo, se definió una estrategia de evaluación y guardado cada 200 pasos, conservando únicamente los dos mejores checkpoints.

El callback personalizado calculó e imprimió la perplejidad (PPL) a partir de la pérdida de validación, permitiendo evaluar en tiempo real la capacidad predictiva del modelo. Durante la ejecución, el entrenamiento procesó el dataset tokenizado dividido en entrenamiento y prueba, aplicando el esquema de causal language modeling sin enmascaramiento de tokens.

In [ ]:
from transformers import TrainerCallback
import math

# ============================
# CALLBACK PERSONALIZADO
# ============================
class PerplexityCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics and "eval_loss" in metrics:
            ppl = math.exp(metrics["eval_loss"])
            print(f"\n🔎 Step {state.global_step}: Eval Loss = {metrics['eval_loss']:.4f} | PPL = {ppl:.2f}")

model_name = "datificate/gpt2-small-spanish"
model, tokenizer = init_model_and_tokenizer(model_name, device=device)

batch_size = 32

training_args = TrainingArguments(
    output_dir="./hf-gpt",
    run_name="poesia-gpt2",
    overwrite_output_dir=True,
    num_train_epochs=20,#20
    learning_rate=3e-5,#3e-5, 5e-5
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay=0.05,#0.01
    warmup_steps=40, #1500, 100
    evaluation_strategy="steps",  
    eval_steps=200,               
    save_steps=200,               
    logging_strategy="steps",
    logging_steps=50,
    logging_first_step=True,
    save_strategy="steps",    
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
    disable_tqdm=False,
    fp16=True,
    gradient_accumulation_steps = 2 #Nuevo
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    callbacks=[PerplexityCallback]   # 👈 Agregamos el callback
)

# ============================
# 4) ENTRENAMIENTO
# ============================
trainer.train()


In [ ]:
def generar_texto(model, tokenizer, prompt="En el silencio de la noche"):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=120,
        num_return_sequences=5,
        temperature=1.1,
        top_k=30,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id   # 🔑 ya definido arriba
    )
    for i, out in enumerate(outputs):
        print(f"=== Poema {i+1} ===")
        print(tokenizer.decode(out, skip_special_tokens=True))
        print()

generar_texto(model, tokenizer, "Había una vez un poeta que soñaba")


Tras el entrenamiento del modelo gpt2-small-spanish con el corpus combinado (poesía, Wikipedia y noticias), los resultados muestran una mejora sustancial frente al entrenamiento realizado únicamente con poesía. A lo largo de las 20 épocas configuradas, el modelo alcanzó un training loss final de 2.94 y un validation loss de 3.31, valores mucho más bajos que en la primera prueba. La métrica de perplejidad (PPL) final se situó en 20.80, lo que representa una reducción significativa en comparación con la PPL de 67.36 obtenida con el dataset exclusivo de poesía.

Tras el entrenamiento del modelo gpt2-small-spanish con el corpus combinado (poesía, Wikipedia y noticias), se evidenció una mejora relevante frente al entrenamiento inicial realizado únicamente con poesía. Durante las 20 épocas configuradas, el modelo alcanzó un training loss final de 2.94 y un validation loss de 3.31, métricas que reflejan un aprendizaje más estable y eficiente. La perplejidad (PPL) final se situó en 20.80, lo que representa una reducción significativa en comparación con el valor de 67.36 obtenido cuando se entrenó únicamente con el corpus de poesía, confirmando el aporte positivo de integrar fuentes textuales más diversas.

Posteriormente se puso a prueba la capacidad generativa del modelo mediante la función generar_texto, configurada con parámetros de muestreo diseñados para promover diversidad y fluidez en los resultados (max_length=120, num_return_sequences=5, temperature=1.1, top_k=30, top_p=0.95). Con el prompt inicial “Había una vez un poeta que soñaba”, el modelo produjo cinco composiciones poéticas distintas, donde los textos generados mostraron avances claros respecto al modelo entrenado únicamente con poesía, manteniendo una estructura más coherente y abordando temáticas más diversas, entre ellas el paso del tiempo, la esperanza, la familia, la vida urbana e incluso referencias históricas o religiosas. No obstante, todavía se observaron repeticiones y fragmentos con menor coherencia semántica, lo que evidencia que, aunque el corpus combinado mejoró de manera notable la calidad de la generación, persisten limitaciones propias del tamaño del dataset y de la arquitectura del modelo.

# Resultados

* La inclusión de Wikipedia y noticias junto con el corpus poético permitió reducir la perplejidad de 67.36 a 20.80, lo que confirma que un dataset más diverso contribuye a mejorar la capacidad de generalización del modelo.

* Los valores finales de training loss (2.94) y validation loss (3.31) reflejan un aprendizaje más controlado y menos propenso al sobreajuste, en comparación con el entrenamiento inicial basado únicamente en poesía.

* Los poemas generados con el corpus combinado presentan estructuras más coherentes y temáticas variadas, lo que indica un mayor dominio del lenguaje y una mejor adaptación a distintos contextos narrativos.

* A pesar de las mejoras, aún se observan repeticiones y fragmentos con baja coherencia semántica, lo que sugiere que el modelo requiere más datos o ajustes adicionales para alcanzar un nivel óptimo en la generación de texto poético, lo que a su vez tambien evidencia la capacidad de computo que se requiere para mejorarlo.

* El experimento confirma que la estrategia de combinar corpus heterogéneos enriquece el vocabulario y la expresividad del modelo, siendo un paso necesario para futuros entrenamientos que busquen resultados más consistentes y creativos en generación de lenguaje natural.